In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType
from pyspark.sql.functions import explode, expr, from_json, col

In [2]:
spark = (SparkSession
         .builder
         .config("spark.streaming.stopGracefullyOnShutdown", True)
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
         .config("spark.sql.shuffle.partitions", 4)
         .master("local[*]")
         .appName("Reading from file").getOrCreate())

In [3]:
spark.conf.set("spark.sql.streaming.schemaInterface", True)
spark

In [4]:
kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "spark_kafka-kafka:29092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

In [5]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
kafka_df_json = kafka_df.withColumn("value", expr("cast(value as string)"))
# kafka_df_json.show()

In [7]:
devices_element_schema = StructType([
    StructField('deviceId', StringType(), True),
    StructField('measure', StringType(), True),
    StructField('status', StringType(), True),
    StructField('temperature', LongType(), True)
])

devices_schema = ArrayType(devices_element_schema, True)

data_schema = StructType([
    StructField('devices', devices_schema, True)
])

json_schema = StructType([
    StructField('customerId', StringType(), True),
    StructField('data', data_schema, True),
    StructField('eventId', StringType(), True),
    StructField('eventOffset', LongType(), True),
    StructField('eventPublisher', StringType(), True),
    StructField('eventTime', StringType(), True),
])

In [8]:
streaming_df = kafka_df_json.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [9]:
streaming_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)



In [10]:
exploded_df = streaming_df.withColumn('devices', explode(streaming_df.data.devices)).drop('data')

In [11]:
exploded_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [12]:
flattened_df = exploded_df.withColumns({
    'device_id': exploded_df.devices.deviceId,
    'measure' : exploded_df.devices.measure,
    'status': exploded_df.devices.status,
    'temperature' : exploded_df.devices.temperature
}).drop('devices')

In [13]:
flattened_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)



In [ ]:
# Running in once/avaiableNow and processingTime mode
# Write the output to console
(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
 .trigger(processingTime='10 seconds')
  #.trigger(avaiableNow=True)
 # .trigger(once=True)
 .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())

In [15]:
# Running in continuous mode
# Using kafka_df DataFrame because unavailable explode function in continuous mode
(kafka_df
 .writeStream
 .format("memory")
 .queryName("kafka_table")
 .outputMode("append")
 .trigger(continuous='10 seconds')
 .option("checkpointLocation", "checkpoint_dir_kafka_4")
 .start()
 .awaitTermination())

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
spark.sql("select * from kafka_table").show()

+--------------------+--------------------+-----------+---------+------+--------------------+-------------+
|                 key|               value|      topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+-----------+---------+------+--------------------+-------------+
|                null|[7B 22 65 76 65 6...|device-data|        0|     0|2024-04-23 18:26:...|            0|
|                null|[7B 22 65 76 65 6...|device-data|        0|     1|2024-04-23 18:26:...|            0|
|                null|[7B 22 65 76 65 6...|device-data|        0|     2|2024-04-24 18:57:...|            0|
|                null|[7B 22 65 76 65 6...|device-data|        0|     3|2024-04-24 18:57:...|            0|
|                null|[7B 22 65 76 65 6...|device-data|        0|     4|2024-04-24 18:57:...|            0|
|                null|[7B 22 65 76 65 6...|device-data|        0|     5|2024-04-24 18:57:...|            0|
|                null|[7B 22

In [ ]:
# (flattened_df
#  .writeStream
#  .format("csv")
#  .outputMode("append")
#  .option("path", "../data/output/events/")        
#  .option("checkpointLocation", "checkpoint_dir")
#  .start()
#  .awaitTermination())